# Ask and answer questions with Queries
Queries are a way to ask summarization questions about meshes. Queries results can be used with Triggers to adapt analysis and visualization actions.  This section shows how to execute queries with Ascent and access query results. See Ascent's [Queries](https://ascent.readthedocs.io/en/latest/Actions/Queries.html) docs for deeper details on Queries.

In [ ]:
# cleanup any old results
!./cleanup.sh

# ascent + conduit imports
import conduit
import conduit.blueprint
import ascent

import numpy as np

# helpers we use to create tutorial data
from ascent_tutorial_jupyter_utils import img_display_width
from ascent_tutorial_jupyter_utils import tutorial_gyre_example

import matplotlib.pyplot as plt

## Query Example 1
### Extracting mesh cycle and entropy of a time varying mesh

In [ ]:
# open ascent
a = ascent.Ascent()
a.open()

# setup actions
actions = conduit.Node()
add_act = actions.append()
add_act["action"] = "add_queries"

# declare a queries to ask some questions
queries = add_act["queries"] 

# add a simple query expression (q1)
queries["q1/params/expression"] = "cycle()"
queries["q1/params/name"] = "cycle"

# add a more complex query expression (q2)
queries["q2/params/expression"] = "entropy(histogram(field('gyre'), num_bins=128))"
queries["q2/params/name"] = "entropy_of_gyre"

# declare a scene to render the dataset
add_scenes = actions.append()
add_scenes["action"] = "add_scenes"
scenes = add_scenes["scenes"] 
scenes["s1/plots/p1/type"] = "pseudocolor"
scenes["s1/plots/p1/field"] = "gyre"
# Set the output file name (ascent will add ".png")
scenes["s1/image_name"] = "out_gyre"

# view our full actions tree
print(actions.to_yaml())

# gyre time varying params
nsteps = 10
time = 0.0
delta_time = 0.5

info = conduit.Node()
for step in range(nsteps):
    # call helper that generates a double gyre time varying example mesh.
    # gyre ref :https://shaddenlab.berkeley.edu/uploads/LCS-tutorial/examples.html
    mesh = tutorial_gyre_example(time)
    
    # update the example cycle
    cycle = 100 + step * 100

    mesh["state/cycle"] = cycle
    print("time: {} cycle: {}".format(time,cycle))
    
    # publish mesh to ascent
    a.publish(mesh)
    
    # update image name
    scenes["s1/image_name"] = "out_gyre_%04d" % step;
    
    # execute the actions
    a.execute(actions)

    # retrieve the info node that contains the query results    
    ts_info = conduit.Node()
    a.info(ts_info)
    
    # add to our running info
    info["expressions"].update(ts_info["expressions"])
    
    # update time
    time = time + delta_time
    
# close ascent
a.close()

In [ ]:
# view the results of the cycle query
print(info["expressions/cycle"].to_yaml())
# Note that query results can be indexed by cycle

In [ ]:
# view the results of the cycle query
print(info["expressions/entropy_of_gyre"].to_yaml())

In [ ]:
# View our rendered images
result_image_files = [ "out_gyre_%04d.png" %i for i in range(nsteps)]
ascent.jupyter.AscentImageSequenceViewer(result_image_files).show()

In [ ]:
# create an array with the entropy values from all 
# cycles
entropy = np.zeros(nsteps)

# get the node that has the time history
gyre = info["expressions/entropy_of_gyre"]
print(gyre)
cycle_names = gyre.child_names()

# transfer conduit data to our summary numpy array
for i in range(gyre.number_of_children()):
    entropy[i] = gyre[i]["value"]

print("Entropy Result")
print(entropy)

# plot the data
plt.plot(cycle_names, entropy)
plt.ylabel('entropy')
plt.xlabel('cycle')
plt.show()